## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.60,74,20,11.50,few clouds
1,1,Mathbaria,BD,22.2876,89.9593,82.60,53,0,2.48,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,39.20,75,0,6.91,clear sky
3,3,Cherskiy,RU,68.7500,161.3000,2.53,86,48,3.44,scattered clouds
4,4,Havelock,US,34.8791,-76.9013,60.01,76,1,10.36,clear sky
5,5,Port Elizabeth,ZA,-33.9180,25.5701,64.40,94,90,17.27,light rain
6,6,Jamestown,US,42.0970,-79.2353,44.60,53,1,3.44,clear sky
7,7,Cape Town,ZA,-33.9258,18.4232,64.00,88,0,10.36,clear sky
8,8,Mataura,NZ,-46.1927,168.8643,57.00,58,100,25.99,overcast clouds
9,9,Lata,PT,40.1629,-8.3327,55.00,93,75,3.44,mist


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.60,74,20,11.50,few clouds
1,1,Mathbaria,BD,22.2876,89.9593,82.60,53,0,2.48,clear sky
10,10,Rikitea,PF,-23.1203,-134.9692,78.12,65,14,17.67,few clouds
15,15,Geraldton,AU,-28.7667,114.6000,77.00,69,90,16.11,overcast clouds
17,17,Kapaa,US,22.0752,-159.3190,75.20,60,90,16.11,overcast clouds
19,19,Hilo,US,19.7297,-155.0900,75.20,73,90,5.75,heavy intensity rain
22,22,Yeppoon,AU,-23.1333,150.7333,75.20,88,90,11.50,light rain
29,29,Butaritari,KI,3.0707,172.7902,82.51,78,92,14.23,overcast clouds
30,30,Tarakan,ID,3.3000,117.6333,81.50,76,96,6.49,overcast clouds
31,31,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,31,2.66,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                201
City                   201
Country                201
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,
1,Mathbaria,BD,82.60,clear sky,22.2876,89.9593,
10,Rikitea,PF,78.12,few clouds,-23.1203,-134.9692,
15,Geraldton,AU,77.00,overcast clouds,-28.7667,114.6000,
17,Kapaa,US,75.20,overcast clouds,22.0752,-159.3190,
19,Hilo,US,75.20,heavy intensity rain,19.7297,-155.0900,
22,Yeppoon,AU,75.20,light rain,-23.1333,150.7333,
29,Butaritari,KI,82.51,overcast clouds,3.0707,172.7902,
30,Tarakan,ID,81.50,overcast clouds,3.3000,117.6333,
31,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

from config import g_key
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

# Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            
        print("Hotel not found... skipping.")
hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Mathbaria,BD,82.60,clear sky,22.2876,89.9593,Shikdar Hotel
10,Rikitea,PF,78.12,few clouds,-23.1203,-134.9692,Pension Maro'i
15,Geraldton,AU,77.00,overcast clouds,-28.7667,114.6000,Broadwater Mariner Resort
17,Kapaa,US,75.20,overcast clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
19,Hilo,US,75.20,heavy intensity rain,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Yeppoon,AU,75.20,light rain,-23.1333,150.7333,Villa Mar Colina
29,Butaritari,KI,82.51,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
30,Tarakan,ID,81.50,overcast clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan
31,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))